In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
data = pd.read_csv('boston.csv')

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
# пустые значения (NULL)
pd.read_sql(
    """
    SELECT 'CRIM' AS Column, COUNT(CRIM) AS Is_empty
    FROM boston
    WHERE CRIM IS NULL
    UNION ALL
    SELECT 'ZN' AS Column, COUNT(ZN) AS Is_empty
    FROM boston
    WHERE ZN IS NULL
    UNION ALL
    SELECT 'INDUS' AS Column, COUNT(INDUS) AS Is_empty
    FROM boston
    WHERE INDUS IS NULL
    UNION ALL
    SELECT 'CHAS' AS Column, COUNT(CHAS) AS Is_empty
    FROM boston
    WHERE CRIM IS CHAS
    UNION ALL
    SELECT 'NOX' AS Column, COUNT(NOX) AS Is_empty
    FROM boston
    WHERE NOX IS NULL
    """,
    con,
)

,Column,Is_empty
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


In [8]:
# пустые значения (NULL) отсутствуют, можно вывести еще значения, которые равны нулю (вариант с IIF)
pd.read_sql(
    """
    SELECT Is_empty.*, Eq_0.Eq_0
    FROM (SELECT 'CRIM' AS Column, SUM(IIF(CRIM IS Null, 1, 0)) AS Is_empty FROM boston
          UNION ALL
          SELECT 'ZN' AS Column, SUM(IIF(CRIM IS Null, 1, 0)) AS Is_empty FROM boston
          UNION ALL
          SELECT 'INDUS' AS Column, SUM(IIF(CRIM IS Null, 1, 0)) AS Is_empty FROM boston
          UNION ALL
          SELECT 'CHAS' AS Column, SUM(IIF(CRIM IS Null, 1, 0)) AS Is_empty FROM boston
          UNION ALL
          SELECT 'NOX' AS Column, SUM(IIF(CRIM IS Null, 1, 0)) AS Is_empty FROM boston) AS Is_empty
          INNER JOIN 
         (SELECT 'CRIM' AS Column, SUM(IIF(CRIM == 0.0, 1, 0)) AS Eq_0 FROM boston
          UNION ALL
          SELECT 'ZN' AS Column, SUM(IIF(ZN == 0.0, 1, 0)) AS Eq_0 FROM boston
          UNION ALL
          SELECT 'INDUS' AS Column, SUM(IIF(INDUS == 0.0, 1, 0)) AS Eq_0 FROM boston
          UNION ALL
          SELECT 'CHAS' AS Column, SUM(IIF(CHAS == 0.0, 1, 0)) AS Eq_0 FROM boston
          UNION ALL
          SELECT 'NOX' AS Column, SUM(IIF(NOX == 0.0, 1, 0)) AS Eq_0 FROM boston) AS Eq_0
          ON Is_empty.Column = Eq_0.Column
    """,
    con,
)

,Column,Is_empty,Eq_0
0,CRIM,0,0
1,ZN,0,372
2,INDUS,0,0
3,CHAS,0,471
4,NOX,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [9]:
pd.read_sql(
    """
    SELECT 'CRIM' AS Column, COUNT(DISTINCT CRIM) AS Count_unique
    FROM boston
    UNION ALL
    SELECT 'ZN' AS Column, COUNT(DISTINCT ZN) AS Count_unique
    FROM boston
    UNION ALL
    SELECT 'INDUS' AS Column, COUNT(DISTINCT INDUS) AS Count_unique
    FROM boston
    UNION ALL
    SELECT 'CHAS' AS Column, COUNT(DISTINCT CHAS) AS Count_unique
    FROM boston
    UNION ALL
    SELECT 'NOX' AS Column, COUNT(DISTINCT NOX) AS Count_unique
    FROM boston
    """,
    con,
)

,Column,Count_unique
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [10]:
# принцип поиска медианы:
# упорядочиваем по возрастанию, нумеруем
# если количество строк (N) четное, берем среднее арифметическое значений строк с номером N/2 и N/2 +1 (4 строки - берем среднее между 2 и 3)
# если количество строк (N) нечетное, берем значение строки с номером N/2, округленное в большую сторону(3 строки - берем значение строки 2)
pd.read_sql(
    """
    SELECT AVG(CRIM) AS Median
    FROM (SELECT CRIM, ROW_NUMBER() OVER(ORDER BY CRIM) AS Str_n, COUNT(CRIM) OVER() AS Str_all FROM boston)
    WHERE (MOD(Str_all, 2) == 0 AND (Str_n == Str_all/2 OR Str_n == Str_all/2 + 1)) OR
          (MOD(Str_all, 2) != 0 AND Str_n == CEIL(Str_all/2))
    """,
    con,
)

,Median
0,0.25651


In [11]:
# Вариант поиска медианы с использованием NTILE
# Берем максимальное значение 1-й половины выборки и минимальное 2-й для расчета среднего значения
# Если первая половина выборки больше второй на одну (нечетное количество записей в таблице) - второе значение делаем пустым (не будет участвовать в расчете)
# Считаем среднее

# Вариант получился громоздким и много делать замен для изменения показателя, по которому считается медиана, поэтому будет использоваться предыдущий вариант

pd.read_sql(
    """
    SELECT Avg(CRIM)
    FROM (SELECT Max(CRIM) AS CRIM
          FROM (SELECT CRIM, NTILE(2) OVER(ORDER BY CRIM) AS NTILE, Count(1) OVER() AS Str_all FROM boston) AS Parts
          WHERE NTILE == 1
          UNION ALL
          SELECT CASE WHEN MOD(Str_all, 2) == 0 THEN Min (CRIM) ELSE NULL END
          FROM (SELECT CRIM, NTILE(2) OVER(ORDER BY CRIM) AS NTILE, Count(1) OVER() AS Str_all FROM boston) AS Parts
          WHERE NTILE == 2)
    """,
    con,
)

,Avg(CRIM)
0,0.25651


In [12]:
# ищем медианы и минимальные значения по нужным столбцам, выводим те, в которых медиана равна минимальному значению
pd.read_sql(
    """
    SELECT *
    FROM (SELECT 'CRIM' AS Column, Min, AVG(CRIM) AS Median
          FROM (SELECT CRIM, ROW_NUMBER() OVER(ORDER BY CRIM) AS Str_n, COUNT(CRIM) OVER() AS Str_all, Min(CRIM) OVER() As Min FROM boston)
          WHERE (MOD(Str_all, 2) == 0 AND (Str_n == Str_all/2 OR Str_n == Str_all/2 + 1)) OR
                (MOD(Str_all, 2) != 0 AND Str_n == CEIL(Str_all/2))
          UNION ALL
          SELECT 'ZN' AS Column, Min, AVG(ZN) AS Median
          FROM (SELECT ZN, ROW_NUMBER() OVER(ORDER BY ZN) AS Str_n, COUNT(ZN) OVER() AS Str_all, Min(ZN) OVER() As Min FROM boston)
          WHERE (MOD(Str_all, 2) == 0 AND (Str_n == Str_all/2 OR Str_n == Str_all/2 + 1)) OR
                (MOD(Str_all, 2) != 0 AND Str_n == CEIL(Str_all/2))
          UNION ALL
          SELECT 'INDUS' AS Column, Min, AVG(INDUS) AS Median
          FROM (SELECT INDUS, ROW_NUMBER() OVER(ORDER BY INDUS) AS Str_n, COUNT(INDUS) OVER() AS Str_all, Min(INDUS) OVER() As Min FROM boston)
          WHERE (MOD(Str_all, 2) == 0 AND (Str_n == Str_all/2 OR Str_n == Str_all/2 + 1)) OR
                (MOD(Str_all, 2) != 0 AND Str_n == CEIL(Str_all/2))
          UNION ALL
          SELECT 'CHAS' AS Column, Min, AVG(CHAS) AS Median
          FROM (SELECT CHAS, ROW_NUMBER() OVER(ORDER BY CHAS) AS Str_n, COUNT(CHAS) OVER() AS Str_all, Min(CHAS) OVER() As Min FROM boston)
          WHERE (MOD(Str_all, 2) == 0 AND (Str_n == Str_all/2 OR Str_n == Str_all/2 + 1)) OR
                (MOD(Str_all, 2) != 0 AND Str_n == CEIL(Str_all/2))
                UNION ALL
          SELECT 'NOX' AS Column, Min, AVG(NOX) AS Median
          FROM (SELECT NOX, ROW_NUMBER() OVER(ORDER BY NOX) AS Str_n, COUNT(NOX) OVER() AS Str_all, Min(NOX) OVER() As Min FROM boston)
          WHERE (MOD(Str_all, 2) == 0 AND (Str_n == Str_all/2 OR Str_n == Str_all/2 + 1)) OR
                (MOD(Str_all, 2) != 0 AND Str_n == CEIL(Str_all/2))) AS All_agg
    WHERE Min == Median
    """,
    con,
)

,Column,Min,Median
0,ZN,0.0,0.0
1,CHAS,0.0,0.0


Вывод: медиана равна минимуму в том случае, если не менее половины значений равны минимуму. В данном случае медиана равна минимуму для столбцов, в которых более половины значений "0" (возможно не заполненые значения указаны как "0")

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [13]:
pd.read_sql(
    """
    SELECT *, (SELECT AVG(RM) FROM (SELECT *, Max(MEDV) OVER() AS Max_price FROM boston) WHERE MEDV == Max_price) AS Avg_room_top_exp,
              (SELECT AVG(RM) FROM (SELECT *, Max(MEDV) OVER() AS Max_price FROM boston) WHERE MEDV == Max_price) - Avg_room_top_cheap AS RM_difference
    FROM (SELECT DISTINCT 25 AS Tot_cheap_flat, AVG(RM) OVER() AS Avg_room_top_cheap
          FROM (SELECT RM FROM Boston ORDER BY MEDV LIMIT 25)
          UNION ALL
          SELECT DISTINCT 50 AS Tot_cheap_flat, AVG(RM) OVER() AS Avg_room_top_cheap
          FROM (SELECT RM FROM Boston ORDER BY MEDV LIMIT 50)
          UNION ALL
          SELECT DISTINCT 100 AS Tot_cheap_flat, AVG(RM) OVER() AS Avg_room_top_cheap
          FROM (SELECT RM FROM Boston ORDER BY MEDV LIMIT 100)
          UNION ALL
          SELECT DISTINCT 200 AS Tot_cheap_flat, AVG(RM) OVER() AS Avg_room_top_cheap
          FROM (SELECT RM FROM Boston ORDER BY MEDV LIMIT 200)
          UNION ALL
          SELECT DISTINCT 300 AS Tot_cheap_flat, AVG(RM) OVER() AS Avg_room_top_cheap
          FROM (SELECT RM FROM Boston ORDER BY MEDV LIMIT 300)) AS Top_cheap
    """,
    con,
)

,Tot_cheap_flat,Avg_room_top_cheap,Avg_room_top_exp,RM_difference
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


Вывод: при увеличении количества топ дешевых домов в выборку начинают попадать более дорогие дома и разница между средним количеством комнат выборки и средним количеством комнат самых дорогих домов сокращается. Чем больше среднее количество комнат в доме, тем выше стоимость

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех знаечений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [14]:
# использованы RANK и DENSE RANK
pd.read_sql(
    """
    SELECT LSTAT, MEDV, RANK, DANSE_RANK
    FROM (SELECT LSTAT, MEDV, Max(MEDV) OVER() AS Max_MEDV, RANK() OVER(ORDER BY LSTAT) AS RANK, dense_rank() OVER(ORDER BY LSTAT) AS DANSE_RANK FROM boston)
    WHERE MEDV == Max_MEDV
    """,
    con,
)

,LSTAT,MEDV,RANK,DANSE_RANK
0,1.73,50.0,1,1
1,1.92,50.0,2,2
2,2.88,50.0,6,6
3,2.96,50.0,8,8
4,2.97,50.0,9,9
5,3.16,50.0,15,14
6,3.26,50.0,17,15
7,3.32,50.0,18,16
8,3.70,50.0,26,23
9,3.73,50.0,27,24


Вывод: в большей части самих дорогих домов очень низкое относительное количество населения с более низким статусом. Несмотря на выбросы, все самые дорогие дома укладываются в 40% домов с самым низким процентом населения с более низким статусом

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [15]:
pd.read_sql(
    """
    SELECT DISTINCT CASE WHEN CHAS == 0.0 THEN "не граничит" ELSE "граничит" END "CHAS",
           Avg(MEDV) OVER(PARTITION BY CHAS) AS Avg_MEDV
    FROM boston
    """,
    con,
)

,CHAS,Avg_MEDV
0,не граничит,22.093843
1,граничит,28.440000


Вывод: у домов, граничащих с рекой, средняя цена значительно ниже

7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [17]:
pd.read_sql(
    """
    SELECT *
    FROM (SELECT CASE WHEN (SELECT Avg(CRIM) FROM boston WHERE CHAS != 0.0) > (SELECT Avg(CRIM) FROM boston WHERE CHAS == 0.0)
                      THEN "CRIM" Else NULL END AS "Column_which_avg_near_river_more"
          UNION ALL
          SELECT CASE WHEN (SELECT Avg(ZN) FROM boston WHERE CHAS != 0.0) > (SELECT Avg(ZN) FROM boston WHERE CHAS == 0.0)
                      THEN "ZN" Else NULL END AS "Column_which_avg_near_river_more"
          UNION ALL
          SELECT CASE WHEN (SELECT Avg(INDUS) FROM boston WHERE CHAS != 0.0) > (SELECT Avg(INDUS) FROM boston WHERE CHAS == 0.0)
                      THEN "INDUS" Else NULL END AS "Column_which_avg_near_river_more"
          UNION ALL
          SELECT CASE WHEN (SELECT Avg(NOX) FROM boston WHERE CHAS != 0.0) > (SELECT Avg(NOX) FROM boston WHERE CHAS == 0.0)
                      THEN "NOX" Else NULL END AS "Column_which_avg_near_river_more") AS avg_near_river_more
    WHERE Column_which_avg_near_river_more IS NOT NULL
    """,
    con,
)

,Column_which_avg_near_river_more
0,INDUS
1,NOX


Вывод: у домов у реки среднее значение акров неторгового бизнеса на город выше (видимо промышленность строилась у реки) и концентрация оксидов азота выше (видимо из-за промышленности у реки)

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [18]:
pd.read_sql(
    """
    WITH INDUS_Percintile AS (SELECT NTILE(10) OVER(ORDER BY INDUS) AS Percentile, INDUS FROM boston),
         NOX_Percintile AS (SELECT NTILE(10) OVER(ORDER BY INDUS) AS Percentile, NOX FROM boston)

    SELECT Perc_INDUS.*, Perc_NOX.Avg_NOX
    FROM (SELECT 10 AS 'Percentile', Avg(INDUS) AS Avg_INDUS FROM INDUS_Percintile WHERE Percentile <= 1
          UNION ALL SELECT 20, Avg (INDUS) FROM INDUS_Percintile WHERE Percentile <= 2
          UNION ALL SELECT 30, Avg (INDUS) FROM INDUS_Percintile WHERE Percentile <= 3
          UNION ALL SELECT 40, Avg (INDUS) FROM INDUS_Percintile WHERE Percentile <= 4
          UNION ALL SELECT 50, Avg (INDUS) FROM INDUS_Percintile WHERE Percentile <= 5
          UNION ALL SELECT 60, Avg (INDUS) FROM INDUS_Percintile WHERE Percentile <= 6
          UNION ALL SELECT 70, Avg (INDUS) FROM INDUS_Percintile WHERE Percentile <= 7
          UNION ALL SELECT 80, Avg (INDUS) FROM INDUS_Percintile WHERE Percentile <= 8
          UNION ALL SELECT 90, Avg (INDUS) FROM INDUS_Percintile WHERE Percentile <= 9
          UNION ALL SELECT 100, Avg (INDUS) FROM INDUS_Percintile WHERE Percentile <= 10) AS Perc_INDUS
          INNER JOIN
         (SELECT 10 AS 'Percentile', Avg(NOX) AS Avg_NOX FROM NOX_Percintile WHERE Percentile <= 1
          UNION ALL SELECT 20, Avg (NOX) FROM NOX_Percintile WHERE Percentile <= 2
          UNION ALL SELECT 30, Avg (NOX) FROM NOX_Percintile WHERE Percentile <= 3
          UNION ALL SELECT 40, Avg (NOX) FROM NOX_Percintile WHERE Percentile <= 4
          UNION ALL SELECT 50, Avg (NOX) FROM NOX_Percintile WHERE Percentile <= 5
          UNION ALL SELECT 60, Avg (NOX) FROM NOX_Percintile WHERE Percentile <= 6
          UNION ALL SELECT 70, Avg (NOX) FROM NOX_Percintile WHERE Percentile <= 7
          UNION ALL SELECT 80, Avg (NOX) FROM NOX_Percintile WHERE Percentile <= 8
          UNION ALL SELECT 90, Avg (NOX) FROM NOX_Percintile WHERE Percentile <= 9
          UNION ALL SELECT 100, Avg (NOX) FROM NOX_Percintile WHERE Percentile <= 10) AS Perc_NOX
          ON Perc_INDUS.Percentile = Perc_NOX.Percentile
    """,
    con,
)

,Percentile,Avg_INDUS,Avg_NOX
0,10,2.016078,0.438378
1,20,2.830686,0.466254
2,30,3.635882,0.460248
3,40,4.344853,0.462896
4,50,5.127098,0.476618
5,60,6.064118,0.483045
6,70,7.634579,0.506446
7,80,8.923424,0.527268
8,90,9.952346,0.543629
9,100,11.136779,0.554695


Вывод: в общем (за исключием 30 перцентиля) при увеличении доли акров неторгового бизнеса на город (примышленности) увеличивается концентрация оксидов азота